## Setup

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats._qmc import Sobol
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel
from scipy.stats import norm
from scipy.optimize import minimize
import warnings
warnings.filterwarnings('ignore')



In [2]:
def upper_confidence_bound(mu, sigma, kappa=2.0):
    """
    Upper Confidence Bound (UCB) acquisition function.
    
    UCB = mean + kappa * std
    
    Parameters:
    -----------
    mu : predicted mean
    sigma : predicted standard deviation
    kappa : exploration parameter (higher = more exploration)
    """
    return mu + kappa * sigma


def expected_improvement(mu, sigma, y_best, xi=0.01):
    """
    Expected Improvement (EI) acquisition function.
    
    EI = E[max(f(x) - f(x_best), 0)]
    
    Parameters:
    -----------
    mu : predicted mean
    sigma : predicted standard deviation  
    y_best : best observed value so far
    xi : exploration parameter
    """
    with np.errstate(divide='warn'):
        improvement = mu - y_best - xi
        Z = improvement / sigma
        ei = improvement * norm.cdf(Z) + sigma * norm.pdf(Z)
        ei[sigma == 0.0] = 0.0
    return ei


In [3]:
def bayesian_optimization_next_point(
        n_dims,
        X_samples,
        y_samples,
        acquisition='ei',
        kappa=5,
        n_sobol=None,
        n_local=None):

    """
    Bayesian Optimization acquisition optimizer - after research about search spaces.
    Parameters
    ----------
    n_dims : int
        Number of dimensions
    X_samples : array (N, D)
        Existing sample inputs
    y_samples : array (N,)
        Existing sample outputs
    acquisition : str
        'ei' or 'ucb'
    n_sobol : int
        Number of Sobol candidate points
    n_local : int
        Number of local optimizations
    kappa: int
        Exploration parameter (higher = more exploration)
    """

    if n_sobol is None:
        n_sobol = min(32000, 2048 * n_dims)
    if n_local is None:
        n_local = 16 * n_dims

    bounds = [(0.0, 1.0) for _ in range(n_dims)]
    print(f"Starting {n_dims}D BO with {len(X_samples)} samples")
    print(f"Current best: {y_samples.max()}")

    # -----------------------------
    # 1. Fit Gaussian Process
    #
    #  alpha is the noise level
    #  n_restarts_optimizer is a hyper-parameter
    #  normalize_y improves stability
    # -----------------------------
    kernel = ConstantKernel(1.0) * RBF(length_scale=0.3)
    gp = GaussianProcessRegressor(
        kernel=kernel,
        alpha=1e-6,
        n_restarts_optimizer=5,
        normalize_y=True
    )
    gp.fit(X_samples, y_samples)

    # -----------------------------
    # 2. Acquisition function
    # -----------------------------
    def acq(X):
        X = np.atleast_2d(X)
        mu, sigma = gp.predict(X, return_std=True)

        if acquisition == 'ei':
            val = expected_improvement(mu, sigma, y_samples.max())
        else:
            val = upper_confidence_bound(mu, sigma, kappa)

        return val

    # -----------------------------
    # 3. Sobol global search
    #
    # Provides a better search than random of the space, but we then run a vectorized evaluation loop and pick the top results to focus optimzing
    # i.e. maximising in local optimization (actually we minimise the -value)
    # -----------------------------
    sobol = Sobol(d=n_dims, scramble=True)

    X_candidates = sobol.random(n_sobol)
    acq_vals = acq(X_candidates)

    # Pick best Sobol points
    best_idx = np.argsort(-acq_vals)[:n_local]
    X_start = X_candidates[best_idx]

    print(f"Sobol candidates: {len(X_candidates)} → local starts: {len(X_start)}")

    # -----------------------------
    # 4. Local optimization
    # -----------------------------
    def objective(x):
        return -acq(x)  # minimize negative acquisition
    best_x = None
    best_val = -np.inf

    for i, x0 in enumerate(X_start):
        result = minimize(
            objective,
            x0,
            bounds=bounds,
            method="L-BFGS-B"
        )

        val = -result.fun
        if val > best_val:
            best_val = val
            best_x = result.x

        print(f"Local search {i+1}/{n_local}, x={result.x}, val={val}, best={best_val:.6f}")
    return best_x, best_val



In [4]:
#Store Results from runs
runResults = {}

In [5]:
def findNextGuess(functionName, f_inputs, f_outputs):

    f_maxInput = np.max(f_inputs, axis=0) #Output per dimension
    f_minInput = np.min(f_inputs, axis=0)
    f_maxOutput = np.max(f_outputs)
    f_minOutput = np.min(f_outputs)
    numDimensions = len(f_maxInput)

    print('********************')
    print(functionName, ' - ', numDimensions, 'D')
    print('********************')
    print(' Input -   Min:',  f_minInput, ', Max: ', f_maxInput )
    print(' Output -  Min:',  f_minOutput, ', Max: ', f_maxOutput )

    print(' First input row of ', f_inputs.shape[0] )
    print(f_inputs[:1])
    print(' First output row:')
    print(f_outputs[0:1])

    nextInputToTest, acqFunctionValue = bayesian_optimization_next_point(n_dims = numDimensions, X_samples = f_inputs, y_samples = f_outputs, acquisition='ei')
    print()
    formattedOutput = np.array2string(nextInputToTest, separator='-', precision=6, floatmode='fixed').strip('[]')
    print('Try: ', formattedOutput)
    print()
    runResults[functionName] = formattedOutput


In [6]:
def collatePreviousSubmissions(functionNo):
    functionIndex = functionNo -1
    assert(functionIndex >= 0)
    assert(functionIndex <= 7)

    with open("submission_results/week1/inputs.txt", "r") as f:
        inputsText = f.read()
    with open("submission_results/week1/outputs.txt", "r") as f:
        resultsText = f.read()

    submissions = eval(inputsText, {"array": np.array})
    results = eval(resultsText, {"np": np})

    return submissions[functionIndex], results[functionIndex]


In [7]:
def combineInitialDataAndSubmissionsToDate(functionNo):
    assert(functionNo >= 1)
    assert(functionNo <= 8)

    initialInputs = np.load(f'initial_data/function_{functionNo}/initial_inputs.npy')
    initialOutputs = np.load(f'initial_data/function_{functionNo}/initial_outputs.npy')

    submissions, results = collatePreviousSubmissions(functionNo)

    combined_Inputs = np.vstack((initialInputs, submissions))
    combined_Outputs = np.concatenate((initialOutputs, [results]))

    assert combined_Inputs.shape[1] == initialInputs.shape[1]
    assert combined_Outputs.shape[0] == combined_Inputs.shape[0]

    return combined_Inputs, combined_Outputs


In [8]:
f1_Inputs, f1_Outputs = combineInitialDataAndSubmissionsToDate(1)
findNextGuess('F1', f1_Inputs, f1_Outputs)

********************
F1  -  2 D
********************
 Input -   Min: [0.0305   0.037348] , Max:  [0.88388983 0.8798981 ]
 Output -  Min: -0.0036060626443634764 , Max:  7.710875114502849e-16
 First input row of  11
[[0.31940389 0.76295937]]
 First output row:
[1.32267704e-79]
Starting 2D BO with 11 samples
Current best: 7.710875114502849e-16
Sobol candidates: 4096 → local starts: 32
Local search 1/32, x=[0.30920083 0.07628798], val=1.1348542015226896e-27, best=0.000000
Local search 2/32, x=[0.51354521 0.9957952 ], val=1.1348542015214268e-27, best=0.000000
Local search 3/32, x=[0.16647985 0.09225087], val=1.1348542015214268e-27, best=0.000000
Local search 4/32, x=[0.32017248 0.73685263], val=1.1348542015214268e-27, best=0.000000
Local search 5/32, x=[0.95031141 0.33173739], val=1.1348542015214268e-27, best=0.000000
Local search 6/32, x=[0.76522601 0.5833785 ], val=1.1348542015214268e-27, best=0.000000
Local search 7/32, x=[0.38019675 0.48862703], val=1.1348542015214268e-27, best=0.000000

In [9]:
f2_Inputs, f2_Outputs = combineInitialDataAndSubmissionsToDate(2)
findNextGuess('F2', f2_Inputs, f2_Outputs)

********************
F2  -  2 D
********************
 Input -   Min: [0.14269907 0.02869772] , Max:  [0.87779099 0.9265642 ]
 Output -  Min: -0.06562362443733738 , Max:  0.6112052157614438
 First input row of  11
[[0.66579958 0.12396913]]
 First output row:
[0.53899612]
Starting 2D BO with 11 samples
Current best: 0.6112052157614438
Sobol candidates: 4096 → local starts: 32
Local search 1/32, x=[0.77868786 0.97076403], val=0.07701469821966286, best=0.077015
Local search 2/32, x=[0.77868812 0.97076421], val=0.07701469822040324, best=0.077015
Local search 3/32, x=[0.77868799 0.97076341], val=0.0770146982162753, best=0.077015
Local search 4/32, x=[0.77868825 0.97076345], val=0.07701469821729667, best=0.077015
Local search 5/32, x=[0.77868815 0.97076414], val=0.07701469822039145, best=0.077015
Local search 6/32, x=[0.7786881 0.9707642], val=0.07701469822040177, best=0.077015
Local search 7/32, x=[0.77868823 0.97076385], val=0.07701469821976209, best=0.077015
Local search 8/32, x=[0.7786885

In [10]:
f3_Inputs, f3_Outputs = combineInitialDataAndSubmissionsToDate(3)
findNextGuess('F3', f3_Inputs, f3_Outputs)

********************
F3  -  3 D
********************
 Input -   Min: [0.04680895 0.028783   0.06608864] , Max:  [0.96599485 0.94135983 0.99088187]
 Output -  Min: -0.3989255131463011 , Max:  -0.034835313350078584
 First input row of  16
[[0.17152521 0.34391687 0.2487372 ]]
 First output row:
[-0.1121222]
Starting 3D BO with 16 samples
Current best: -0.034835313350078584
Sobol candidates: 6144 → local starts: 48
Local search 1/48, x=[0.37145202 0.41306468 0.46148429], val=0.022664775043771675, best=0.022665
Local search 2/48, x=[0.37144562 0.4130468  0.46149015], val=0.022664774959069635, best=0.022665
Local search 3/48, x=[0.37145242 0.4130661  0.46148276], val=0.022664775044703066, best=0.022665
Local search 4/48, x=[0.37145332 0.41306804 0.46148477], val=0.022664775040692926, best=0.022665
Local search 5/48, x=[0.37144863 0.41306337 0.46148261], val=0.022664775039991432, best=0.022665
Local search 6/48, x=[0.37146333 0.41307246 0.46148208], val=0.022664775001427225, best=0.022665
Loc

In [11]:
f4_Inputs, f4_Outputs = combineInitialDataAndSubmissionsToDate(4)
findNextGuess('F4', f4_Inputs, f4_Outputs)

********************
F4  -  4 D
********************
 Input -   Min: [0.03782483 0.0062504  0.04218635 0.08151656] , Max:  [0.98562189 0.91959232 0.93917791 0.99948256]
 Output -  Min: -32.625660215962455 , Max:  -4.025542281908162
 First input row of  31
[[0.89698105 0.72562797 0.17540431 0.70169437]]
 First output row:
[-22.10828779]
Starting 4D BO with 31 samples
Current best: -4.025542281908162
Sobol candidates: 8192 → local starts: 64
Local search 1/64, x=[0.43157982 0.41160901 0.37978851 0.41298084], val=2.929180262594529, best=2.929180
Local search 2/64, x=[0.43157929 0.41160853 0.37978826 0.4129808 ], val=2.929180262557681, best=2.929180
Local search 3/64, x=[0.43157972 0.41160903 0.37978838 0.41298063], val=2.929180262592073, best=2.929180
Local search 4/64, x=[0.43157969 0.41160904 0.37978829 0.41298082], val=2.929180262593467, best=2.929180
Local search 5/64, x=[0.43157984 0.41160916 0.3797884  0.41298074], val=2.929180262594621, best=2.929180
Local search 6/64, x=[0.4315797

In [12]:
f5_Inputs, f5_Outputs = combineInitialDataAndSubmissionsToDate(5)
findNextGuess('F5', f5_Inputs, f5_Outputs)

********************
F5  -  4 D
********************
 Input -   Min: [0.11987923 0.03819337 0.08894684 0.07288048] , Max:  [0.924901   0.86254031 0.87948418 0.9576439 ]
 Output -  Min: 0.1129397953712203 , Max:  1088.8596181962705
 First input row of  21
[[0.19144708 0.03819337 0.60741781 0.41458414]]
 First output row:
[64.44343986]
Starting 4D BO with 21 samples
Current best: 1088.8596181962705
Sobol candidates: 8192 → local starts: 64
Local search 1/64, x=[0.29839754 0.84080762 0.97943196 0.91888107], val=105.91856915098421, best=105.918569
Local search 2/64, x=[0.29839756 0.84080765 0.97943194 0.91888109], val=105.91856915098303, best=105.918569
Local search 3/64, x=[0.29839754 0.84080764 0.97943197 0.91888105], val=105.91856915098809, best=105.918569
Local search 4/64, x=[0.29839756 0.84080766 0.97943195 0.91888105], val=105.91856915098649, best=105.918569
Local search 5/64, x=[0.29839758 0.8408076  0.97943197 0.91888104], val=105.91856915097445, best=105.918569
Local search 6/64,

In [13]:
f6_Inputs, f6_Outputs = combineInitialDataAndSubmissionsToDate(6)
findNextGuess('F6', f6_Inputs, f6_Outputs)


********************
F6  -  5 D
********************
 Input -   Min: [0.02173531 0.11440374 0.0165229  0.04561319 0.003667  ] , Max:  [0.95773967 0.93187122 0.989507   0.96165559 0.89281919]
 Output -  Min: -2.5711696316081234 , Max:  -0.7142649478202404
 First input row of  21
[[0.7281861  0.15469257 0.73255167 0.69399651 0.05640131]]
 First output row:
[-0.71426495]
Starting 5D BO with 21 samples
Current best: -0.7142649478202404
Sobol candidates: 10240 → local starts: 80
Local search 1/80, x=[0.40074046 0.36511808 0.46532783 0.76545948 0.14311944], val=0.28997872329902497, best=0.289979
Local search 2/80, x=[0.40074046 0.36511744 0.46532894 0.76546071 0.14311784], val=0.2899787233108315, best=0.289979
Local search 3/80, x=[0.40074143 0.36511646 0.46532981 0.76546016 0.14311708], val=0.28997872330457375, best=0.289979
Local search 4/80, x=[0.40074051 0.36511728 0.46532895 0.76546098 0.143118  ], val=0.2899787233109185, best=0.289979
Local search 5/80, x=[0.40074041 0.36511667 0.46532

In [14]:
f7_Inputs, f7_Outputs = combineInitialDataAndSubmissionsToDate(7)
findNextGuess('F7', f7_Inputs, f7_Outputs)

********************
F7  -  6 D
********************
 Input -   Min: [0.011684   0.01181284 0.00363456 0.07365919 0.01494418 0.05109986] , Max:  [0.94245084 0.9246939  0.92457051 0.96101714 0.9986547  0.95101392]
 Output -  Min: 0.0027014650245082332 , Max:  1.3649683044991994
 First input row of  31
[[0.27262382 0.32449536 0.89710881 0.83295115 0.15406269 0.79586362]]
 First output row:
[0.6044327]
Starting 6D BO with 31 samples
Current best: 1.3649683044991994
Sobol candidates: 12288 → local starts: 96
Local search 1/96, x=[0.03159857 0.44435868 0.27994597 0.17309881 0.36660726 0.73198379], val=0.02016631292875673, best=0.020166
Local search 2/96, x=[0.03161923 0.44435504 0.27996506 0.17309781 0.36661255 0.73198847], val=0.02016631307112674, best=0.020166
Local search 3/96, x=[0.03161757 0.44436055 0.27995595 0.1730888  0.36660637 0.73198822], val=0.020166313181481148, best=0.020166
Local search 4/96, x=[0.03160683 0.4443615  0.27995826 0.17309338 0.36661106 0.7319873 ], val=0.020166

In [15]:
f8_Inputs, f8_Outputs = combineInitialDataAndSubmissionsToDate(8)
findNextGuess('F8', f8_Inputs, f8_Outputs)

********************
F8  -  8 D
********************
 Input -   Min: [0.00907698 0.0034195  0.02292868 0.00904349 0.00964888 0.02211341
 0.03590888 0.04195607] , Max:  [0.98594539 0.97397979 0.9988855  0.90298577 0.986902   0.99024381
 0.99291449 0.9887551 ]
 Output -  Min: 5.5921933895401965 , Max:  9.598482002566342
 First input row of  41
[[0.60499445 0.29221502 0.90845275 0.35550624 0.20166872 0.57533801
  0.31031095 0.73428138]]
 First output row:
[7.3987211]
Starting 8D BO with 41 samples
Current best: 9.598482002566342
Sobol candidates: 16384 → local starts: 128
Local search 1/128, x=[0.11991892 0.15758836 0.07379771 0.         0.94603349 0.23382186
 0.04595025 0.5690074 ], val=0.782437505483501, best=0.782438
Local search 2/128, x=[0.11992053 0.15758752 0.07380025 0.         0.94603338 0.23382093
 0.04594826 0.569008  ], val=0.7824375054377153, best=0.782438
Local search 3/128, x=[0.1199178  0.15759518 0.07379783 0.         0.94603144 0.23382182
 0.04595202 0.56901225], val=0.7

In [16]:
print (runResults)

{'F1': '0.309201-0.076288', 'F2': '0.778688-0.970764', 'F3': '0.371452-0.413066-0.461483', 'F4': '0.431580-0.411609-0.379788-0.412981', 'F5': '0.298398-0.840808-0.979432-0.918881', 'F6': '0.400740-0.365117-0.465329-0.765461-0.143118', 'F7': '0.031617-0.444361-0.279956-0.173089-0.366607-0.731988', 'F8': '0.119919-0.157590-0.073797-0.000000-0.946033-0.233821-0.045954-0.569009'}
